<a href="https://colab.research.google.com/github/RianAdrian29/ds_NLP/blob/main/FIX_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install evaluate
!pip install transformers datasets torch wordcloud
!pip install rouge_score

# Import libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq
from tqdm import tqdm
from datasets import Dataset
import evaluate

tqdm.pandas()
plt.style.use('ggplot')

# Setup folder output
folders = ['outputs/eda', 'outputs/preprocessed_data', 'outputs/models', 'outputs/evaluation']
for folder in folders:
    os.makedirs(folder, exist_ok=True)
print("Folder struktur siap.")

# Load dataset
url = "https://raw.githubusercontent.com/nadiyapuspitsari20/datasetjamet/main/jamet_4000_pasti.csv"
data = pd.read_csv(url)
data.rename(columns={"informal": "slang", "formal": "formal"}, inplace=True)
data.dropna(inplace=True)
print(f"Dataset dimuat: {data.shape[0]} data")

# Preprocessing
def clean(text):
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Create the clean columns *before* using them in EDA
data['slang_clean'] = data['slang'].apply(clean)
data['formal_clean'] = data['formal'].apply(clean)


# EDA
data['slang_len'] = data['slang_clean'].apply(lambda x: len(str(x).split()))
data['formal_len'] = data['formal_clean'].apply(lambda x: len(str(x).split()))

plt.figure(figsize=(12, 6))
sns.histplot(data['slang_len'], color='red', kde=True, label='Slang')
sns.histplot(data['formal_len'], color='blue', kde=True, label='Formal')
plt.legend()
plt.title('Distribusi Panjang Kalimat')
plt.savefig('outputs/eda/text_length_distribution.png')
plt.close()

# Wordcloud
def save_wordcloud(texts, filename):
    all_words = ' '.join(texts)
    wc = WordCloud(width=800, height=400, background_color='white').generate(all_words)
    wc.to_file(filename)

save_wordcloud(data['slang_clean'], 'outputs/eda/slang_wordcloud.png')
save_wordcloud(data['formal_clean'], 'outputs/eda/formal_wordcloud.png')


slang2formal_df = pd.DataFrame({
    'input': "translate informal to formal: " + data['slang_clean'],
    'target': data['formal_clean']
})

formal2slang_df = pd.DataFrame({
    'input': "translate formal to informal: " + data['formal_clean'],
    'target': data['slang_clean']
})

# Gabungkan untuk training dua arah
combined_df = pd.concat([slang2formal_df, formal2slang_df], ignore_index=True)
combined_df.to_csv("outputs/preprocessed_data/cleaned_bidirectional_data.csv", index=False)

# Split dataset
train_df, val_df = train_test_split(combined_df, test_size=0.1, random_state=42)
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)

# Load tokenizer dan model IndoT5
model_name = "Wikidepia/IndoT5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tokenisasi
def tokenize(batch):
    source = tokenizer(batch["input"], padding="max_length", truncation=True, max_length=64)
    target = tokenizer(batch["target"], padding="max_length", truncation=True, max_length=64)
    labels = target["input_ids"]
    labels = [
        [(label if label != tokenizer.pad_token_id else -100) for label in label_seq]
        for label_seq in labels
    ]
    source["labels"] = labels
    return source

train_tokenized = train_ds.map(tokenize, batched=True)
val_tokenized = val_ds.map(tokenize, batched=True)

# Training args
args = Seq2SeqTrainingArguments(
    output_dir="outputs/models",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir="outputs/models/logs"
)

collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    data_collator=collator,
    tokenizer=tokenizer,
)

# Train
print("Melatih model dua arah...")
trainer.train()
model.save_pretrained("outputs/models")
tokenizer.save_pretrained("outputs/models")
print("Model tersimpan.")

# Evaluasi
print("Evaluasi model...")
preds = trainer.predict(val_tokenized)

predictions_to_decode = np.copy(preds.predictions)
predictions_to_decode[predictions_to_decode == -100] = tokenizer.pad_token_id

decoded_preds = tokenizer.batch_decode(predictions_to_decode, skip_special_tokens=True)

if isinstance(preds.label_ids, np.ndarray):
    labels_to_decode = np.copy(preds.label_ids)
    labels_to_decode[labels_to_decode == -100] = tokenizer.pad_token_id
    decoded_labels = tokenizer.batch_decode(labels_to_decode, skip_special_tokens=True)
else:
    decoded_labels = tokenizer.batch_decode(preds.label_ids, skip_special_tokens=True)

eval_df = pd.DataFrame({
    'input': val_df['input'].values,
    'label': decoded_labels,
    'prediction': decoded_preds
})
eval_df.to_csv("outputs/evaluation/evaluation_results.csv", index=False)

metric = evaluate.load("rouge")
result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
with open("outputs/evaluation/rouge_scores.txt", "w") as f:
    f.write(str(result))
print("Evaluasi selesai.")

# Fungsi Prediksi
def predict(text, mode="slang2formal"):
    if mode == "slang2formal":
        prompt = "translate informal to formal: " + text
    elif mode == "formal2slang":
        prompt = "translate formal to informal: " + text
    else:
        raise ValueError("Mode tidak dikenali.")
    inputs = tokenizer(prompt, return_tensors="pt", max_length=64, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    output = model.generate(**inputs, max_length=64, num_beams=4)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Interface interaktif
while True:
    mode = input("Pilih mode translasi [1 = Slang → Formal, 2 = Formal → Slang, exit = keluar]: ").strip()
    if mode.lower() == "exit":
        break
    elif mode == "1":
        text_input = input("Masukkan kalimat *gaul*: ")
        result = predict(text_input, mode="slang2formal")
        print(f"Hasil terjemahan ke *formal*: {result}")
    elif mode == "2":
        text_input = input("Masukkan kalimat *formal*: ")
        result = predict(text_input, mode="formal2slang")
        print(f"Hasil terjemahan ke *gaul*: {result}")
    else:
        print("Pilihan tidak valid. Silakan masukkan 1, 2, atau 'exit'.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 857.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/696 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/777k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

Melatih model dua arah...


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nadiyaforcollege (nadiyaforcollege-sriwijaya-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss


In [ ]:
log_history = trainer.state.log_history

train_loss = []
eval_loss = []
rougeL = []
epochs = []

for log in log_history:
    if 'loss' in log and 'epoch' in log:
        train_loss.append(log['loss'])
        epochs.append(log['epoch'])
    if 'eval_loss' in log:
        eval_loss.append(log['eval_loss'])
    if 'eval_rougeL' in log:
        rougeL.append(log['eval_rougeL'])

# Plot Train & Eval Loss
plt.figure(figsize=(10, 5))
plt.plot(epochs[:len(train_loss)], train_loss, label="Train Loss", marker='o')
plt.plot(epochs[:len(eval_loss)], eval_loss, label="Eval Loss", marker='s')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training dan Evaluation Loss per Epoch")
plt.legend()
plt.grid(True)
plt.savefig("outputs/evaluation/loss_per_epoch.png")
plt.show()

# Plot ROUGE-L
if rougeL:
    plt.figure(figsize=(10, 5))
    plt.plot(epochs[:len(rougeL)], rougeL, label="ROUGE-L", marker='^', color='green')
    plt.xlabel("Epoch")
    plt.ylabel("ROUGE-L Score")
    plt.title("Performa ROUGE-L per Epoch")
    plt.legend()
    plt.grid(True)
    plt.savefig("outputs/evaluation/rougeL_per_epoch.png")
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Buat label biner: 1 = match, 0 = mismatch
eval_df['exact_match'] = eval_df.apply(lambda row: 1 if row['label'].strip() == row['prediction'].strip() else 0, axis=1)

tp = eval_df['exact_match'].sum()
fn = len(eval_df) - tp

conf_matrix = np.array([[tp, fn]])
print(f"Confusion Matrix-Style Summary:{'-'*40}")
print(f"{'':<20} | {'Benar':<8} | {'Salah':<8}")
print(f"{'-'*20}-+{'-'*9}+{'-'*9}")
print(f"{'Total Prediksi':<20} | {tp:<8} | {fn:<8}")

plt.figure(figsize=(5, 5))
plt.pie([tp, fn], labels=["Benar", "Salah"], autopct="%1.1f%%", colors=["green", "red"])
plt.title("Evaluasi Prediksi Translasi (Exact Match)")
plt.savefig("outputs/evaluation/exact_match_piechart.png")
plt.show()